In [ ]:
!wget https://github.com/phatjkk/data/raw/main/LLM/translate/train.en.txt
!wget https://github.com/phatjkk/data/raw/main/LLM/translate/train.vi.txt

--2023-08-19 07:26:52--  https://github.com/phatjkk/data/raw/main/LLM/translate/train.en.txt
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/phatjkk/data/main/LLM/translate/train.en.txt [following]
--2023-08-19 07:26:52--  https://raw.githubusercontent.com/phatjkk/data/main/LLM/translate/train.en.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13603614 (13M) [text/plain]
Saving to: ‘train.en.txt’

train.en.txt        100%[===================>]  12.97M  --.-KB/s    in 0.04s   

2023-08-19 07:26:53 (309 MB/s) - ‘train.en.txt’ saved [13603614/13603614]

--2023-08-19 07:26:53--  https://gith

In [ ]:
! pip install datasets transformers accelerate==0.21.0 sacrebleu git+https://github.com/huggingface/peft.git

  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-usov86g0
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-usov86g0
  Resolved https://github.com/huggingface/peft.git to commit 6c44096c7b8d55a2ecf24be9bc68393467e1584a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.6 MB/s eta 0:

In [ ]:
from datasets import load_dataset
ds_en = load_dataset("text", data_files={"train": ["train.en.txt"]})
ds_vi = load_dataset("text", data_files={"train": ["train.vi.txt"]})
ds_vi["train"] = ds_vi["train"].add_column("labels", ds_en["train"]["text"])
ds = ds_vi["train"]
ds

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['text', 'labels'],
    num_rows: 133317
})

In [ ]:
ds["labels"][2]

'I &apos;d like to talk to you today about the scale of the scientific effort that goes into making the headlines you see in the paper .'

In [ ]:
def preprocess(text):
  text = text.replace('&apos;', "'")
  text = text.replace('&quot;', "'")
  text = text.replace('&#91;', "(")
  text = text.replace('&#93;', ")")
  return text

def clean_data(sample):
  sample["text"] = preprocess(sample["text"])
  sample["labels"] = preprocess(sample["labels"])
  return sample

# Cleaning
ds = ds.map(clean_data)

Map:   0%|          | 0/133317 [00:00<?, ? examples/s]

In [ ]:
ds["labels"][2]

"I 'd like to talk to you today about the scale of the scientific effort that goes into making the headlines you see in the paper ."

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large")

In [ ]:
max_input_length = 128
max_target_length = 128

def tokeni(examples):
    #Anh -> Viet
    # model_inputs = tokenizer(examples["labels"],
    #                          text_target=examples["text"],
    #                          max_length=128,
    #                          truncation=True)
    #Viet -> Anh
    model_inputs = tokenizer(examples["text"],
                          text_target=examples["labels"],
                          max_length=128,
                          truncation=True)
    return model_inputs

In [ ]:
tokenized_datasets = ds.map(tokeni,batched=True)

Map:   0%|          | 0/133317 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

Dataset({
    features: ['text', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 133317
})

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
tokenized_datasets.push_to_hub("phatjk/ds_translate_vi_en_bart")
# tokenized_datasets.push_to_hub("phatjk/ds_translate_en_vi_bart")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/134 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

# Training

In [ ]:
from datasets import load_dataset
dataset_train = load_dataset("phatjk/ds_translate_vi_en_bart",split="train[:90%]")
dataset_test = load_dataset("phatjk/ds_translate_vi_en_bart",split="train[90%:]")
# dataset_train = load_dataset("phatjk/ds_translate_en_vi_bart",split="train[:90%]")
# dataset_test = load_dataset("phatjk/ds_translate_en_vi_bart",split="train[90%:]")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/133317 [00:00<?, ? examples/s]

In [ ]:
tokenizer.decode(dataset_train[2]["labels"])

"<s>I 'd like to talk to you today about the scale of the scientific effort that goes into making the headlines you see in the paper.</s>"

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

# model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large")
model = AutoModelForSeq2SeqLM.from_pretrained("phatjk/vi_en_bart_translate")
# model = AutoModelForSeq2SeqLM.from_pretrained("phatjk/en_vi_bart_translate")

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
from datasets import load_metric
metric = load_metric("sacrebleu")

<ipython-input-6-523d71b03492>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


In [ ]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # Kiểm tra preds có phải tuple
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
batch_size = 25
model_name = "bart-translate"
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    # report_to="wandb",
    # run_name="bart-vi-en"
)

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Currently logged in as: autoittutorial1234. Use `wandb login --relogin` to force relogin


You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,1.704700,1.820606,13.328700,16.997300


TrainOutput(global_step=4800, training_loss=1.489127426147461, metrics={'train_runtime': 4311.8466, 'train_samples_per_second': 27.827, 'train_steps_per_second': 1.113, 'total_flos': 3.249737365131264e+16, 'train_loss': 1.489127426147461, 'epoch': 1.0})

In [ ]:
model.push_to_hub("phatjk/vi_en_bart_translate")
# model.push_to_hub("phatjk/en_vi_bart_translate")

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/phatjk/vi_en_bart_translate/commit/9b86bf14ded46c5cf65e532850a4d2cf8cd82163', commit_message='Upload BartForConditionalGeneration', commit_description='', oid='9b86bf14ded46c5cf65e532850a4d2cf8cd82163', pr_url=None, pr_revision=None, pr_num=None)

# **RUN(CHẠY TỪ ĐÂY)**

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.2 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large")

In [ ]:
!gsutil cp gs://vietai_public/best_vi_translation/v1/tst2013.* /content/data/

CommandException: Destination URL must name a directory, bucket, or bucket
subdirectory for the multiple source form of the cp command.


In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained("phatjk/vi_en_bart_translate")
# model = AutoModelForSeq2SeqLM.from_pretrained("phatjk/en_vi_bart_translate")

In [ ]:
model.num_parameters()

406291456

In [ ]:
!pip install datasets sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 18.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.1 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=ba181f7d083acea421fc163cb8c13feeb7976d42e447c8b07b191adf2ed3c3c2
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [ ]:
!gsutil cp gs://vietai_public/best_vi_translation/v1/tst2013.* /content/

Copying gs://vietai_public/best_vi_translation/v1/tst2013.en...
Copying gs://vietai_public/best_vi_translation/v1/tst2013.en-vi.json...
Copying gs://vietai_public/best_vi_translation/v1/tst2013.vi...
| [3 files][762.3 KiB/762.3 KiB]                                                
Operation completed over 3 objects/762.3 KiB.                                    


In [ ]:
def preprocess(text):
  text = text.replace('&apos;', "'")
  text = text.replace('&quot;', "'")
  text = text.replace('&#91;', "(")
  text = text.replace('&#93;', ")")
  return text
with open("tst2013.vi", 'r') as f:
    references = [[preprocess(line.strip()),] for line in f]

In [ ]:
from transformers import pipeline

translator = pipeline("translation", model=model.to("cuda"),tokenizer=tokenizer,device="cuda")
translator("Tôi đang hạnh phúc")[0]["translation_text"]


"I 'm happy ."

In [ ]:
predictions = translator(references,max_length=128, batch_size=25)

Your input_length: 195 is bigger than 0.9 * max_length: 128. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 276 is bigger than 0.9 * max_length: 128. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 189 is bigger than 0.9 * max_length: 128. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 280 is bigger than 0.9 * max_length: 128. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 225 is bigger than 0.9 * max_length: 128. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 264 is bigger than 0.9 * max_length: 128. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 367 is bigger than 0.9 * max_length: 128. You

In [ ]:
with open("tst2013.en", 'r') as f:
  targets = [[preprocess(line.strip()),] for line in f]

In [ ]:
predictions[0]

[{'translation_text': "When I was a kid , I thought that North Korea was the best country in the world , and I used to sing , ' We have nothing to be ashamed of . '"}]

In [ ]:
targets[0]

["When I was little , I thought my country was the best on the planet , and I grew up singing a song called ' Nothing To Envy . '"]

In [ ]:
len(predictions),len(targets)

(1268, 1268)

In [ ]:
predictions[0][0]

{'translation_text': "When I was a kid , I thought that North Korea was the best country in the world , and I used to sing , ' We have nothing to fear . '"}

In [ ]:
with open('bart_vi_en_13_predict.txt', 'w') as f:
  for item in predictions:
    f.write(item[0]["translation_text"] + '\n')


In [ ]:
with open("bart_vi_en_13_predict.txt", 'r') as f:
  predicts = [[preprocess(line.strip()),] for line in f]

In [ ]:
from datasets import load_metric
# Compute sacrebleu
metric = load_metric("sacrebleu")
print('sacreBLEU: %.2f' % metric.compute(predictions=predicts, references=targets)['score'])

sacreBLEU: 25.78


In [ ]:
translator("Hạnh phúc")

[{'translation_text': 'Happy .'}]

In [ ]:
translator("Hôm nay em đến trường")

Your input_length: 21 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


[{'translation_text': 'Today I went to school .'}]

In [ ]:
translator("Tôi thích lịch sử")

[{'translation_text': 'I love history .'}]

In [ ]:
translator("Một con chim đang bay")

[{'translation_text': 'A bird is flying .'}]

In [ ]:
translator("Bố tôi đang ăn sáng",max_length=128)

[{'translation_text': 'My father was eating breakfast .'}]

In [ ]:
translator("Tôi đang ngồi chơi game",max_length=500)

[{'translation_text': "I 'm sitting here playing a game ."}]

In [ ]:
# from google.colab import runtime
# runtime.unassign()